In [13]:
import numpy as np
from scipy import stats
import math

### For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like.



- **Is the website redesign any good?**

>**H<sub>o</sub>:** website redesign is not good, click rate has no significant change
>
>>False Negative **(type II error)** Click rate has not changed, but another metric has improved that we're not tracking
>
>**H<sub>a</sub>:** website redesign is good, click rate has significantly improved
>
>>False Positive **(type I error)** Click rate has signifaicantly improved but not due to website redesign


- **Is our television ad driving more sales?**

>**H<sub>o</sub>:** Ad has no effect on sales, no significant increase
>
>>False Negative **(type II error)** sales have not risen, but due to other causes
>
>**H<sub>a</sub>:** Ad has boosted sales
>
>>False Positive **(type I error)** Sales have risen, but due to other causes

- **Has the network latency gone up since we switched internet service providers?**

>**H<sub>o</sub>:** Network latency has not been affected by new internet provider
>
>>False Negative **(type II error)** latency has risen but measurment device is not working
>
>**H<sub>a</sub>:** Network latency has risen significantly
>
>>False Positive **(type I error)** latency has not risen but measurment device is not working

# T-tests

- Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.

In [24]:
#metrics
dist_1 = stats.norm(90, 15).rvs(40)
dist_2 = stats.norm(100,20).rvs(50)

n1 = 40
n2 = 50

μ1 = 90
μ2 = 100

σ1 = 15
σ2 = 20

alpha = .05

In [11]:
#two sample t-test
#h0: time1 = time2
#ha: time1 ≠ time2
deg_f = n1 + n2 - 2
σ_pool = math.sqrt(((n1 - 1)*(σ1 ** 2) + (n2 - 1)*(σ2 ** 2)) / (deg_f))

t = (μ1 - μ2) / (σ_pool * math.sqrt(1/n1 + 1/n2))
t

-2.6252287036468456

In [17]:
p = stats.t(deg_f).cdf(t)
p

0.005104926224619695

In [25]:
stats.ttest_ind(dist_1, dist_2)

Ttest_indResult(statistic=-1.3656244902171626, pvalue=0.1755373202718037)